In [572]:
import numpy as np
import pandas as pd

rankings = pd.read_csv('./worldCupData/fifaRanking2020-10-06.csv', encoding='windows-1252')
matches = pd.read_csv('./worldCupData/results.csv', encoding='windows-1252')
groups = pd.read_csv('./worldCupData/Qatar2022-teams.csv')

In [573]:
rankings = rankings.replace({"IR Iran": "Iran"})
rankings = rankings.replace({"Korea Republic": "South Korea"})
rankings['rank_date'] = pd.to_datetime(rankings['rank_date'])
matches['date'] = pd.to_datetime(matches['date'])
matches = matches.replace({"United States": "USA"})

In [574]:
rankings.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,164,Belize,BLZ,110.0,92.0,-3,CONCACAF,2013-01-17
1,163,Hong Kong,HKG,111.0,112.0,0,AFC,2013-01-17
2,120,Kuwait,KUW,276.0,281.0,3,AFC,2013-01-17
3,153,Faroe Islands,FRO,151.0,151.0,0,UEFA,2013-01-17
4,152,Palestine,PLE,154.0,152.0,0,AFC,2013-01-17


In [575]:
country_list = groups['Team'].values.tolist()
country_list = sorted(country_list)
# country_list

In [576]:
avgRanking = rankings.groupby(['country_full']).mean()
avgRanking = avgRanking.reset_index()

In [577]:
matches.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'country', 'neutral'],
      dtype='object')

In [578]:
# Different number of data
home_offense = matches.groupby(['home_team']).mean()['home_score'].fillna(0)
len(home_offense)

283

In [579]:
# Different number of data
away_offense = matches.groupby(['away_team']).mean()['away_score'].fillna(0)
len(away_offense)

278

Different number of data <br>
so extract countries which join the 2022 Qatar World Cup <br><br>
Countries: ['Senegal',
 'Qatar',
 'Netherlands',
 'Ecuador',
 'Iran',
 'England',
 'USA',
 'Wales',
 'Argentina',
 'Saudi Arabia',
 'Mexico',
 'Poland',
 'Denmark',
 'Tunisia',
 'France',
 'Australia',
 'Germany',
 'Japan',
 'Spain',
 'Costa Rica',
 'Morocco',
 'Croatia',
 'Belgium',
 'Canada',
 'Switzerland',
 'Cameroon',
 'Brazil',
 'Serbia',
 'Uruguay',
 'South Korea',
 'Portugal',
 'Ghana']

In [580]:
home = home_offense.to_frame().reset_index()
away = away_offense.to_frame().reset_index()

In [581]:
home = home[home['home_team'].isin(country_list)].reset_index()
home = home.drop(['index'], axis = 1)
#home

In [582]:
away = away[away['away_team'].isin(country_list)].reset_index()
away = away.drop(['index'], axis = 1)
# away

In [583]:
wc_score = pd.DataFrame()
wc_score['country_name'] = country_list
wc_score['offense_score'] = round((home['home_score'] * 0.3 + away['away_score'] * 0.7), 2) # Most gmaes are away game so I weighted more in away_score
# wc_score

In [584]:
home_diffense = matches.groupby(['home_team']).mean()['away_score'].fillna(0)
away_diffense = matches.groupby(['away_team']).mean()['home_score'].fillna(0)
home = home_diffense.to_frame().reset_index()
away = away_diffense.to_frame().reset_index()
home = home[home['home_team'].isin(country_list)].reset_index()
home = home.drop(['index'], axis = 1)
away = away[away['away_team'].isin(country_list)].reset_index()
away = away.drop(['index'], axis = 1)
wc_score['diffense_score'] = round(home['away_score'] * 0.3 + away['home_score'] * 0.7, 2) # most games are away
wc_score['GD'] = (wc_score['offense_score'] - wc_score['diffense_score']) # Goals Difference
# wc_score

In [585]:
avgRanking = avgRanking[['country_full', 'rank']]
avgRanking.head()

,country_full,rank
0,Afghanistan,146.207921
1,Albania,56.524752
2,Algeria,37.930693
3,American Samoa,190.396040
4,Andorra,173.128713


In [586]:
avgRank = avgRanking[avgRanking['country_full'].isin(country_list)].reset_index()
avgRank = avgRank.drop(['index'], axis = 1)
# avgRank

In [587]:
matches['score_difference_home'] = matches['home_score'] - matches['away_score']
matches['score_difference_away'] = matches['away_score'] - matches['home_score']
matches['home win'] = ((matches['score_difference_home'] > 0) & (matches['tournament'] != 'Friendly'))
matches['away win'] = ((matches['score_difference_away'] > 0) & (matches['tournament'] != 'Friendly'))
matches = matches[(matches['home_team'].isin(country_list)) | matches['away_team'].isin(country_list)]
# matches

In [588]:
winRate = {'country' : [], 'winrate': []}
for i in country_list:
    count = matches[(matches['home_team'] == i) | (matches['away_team'] == i) == True]
    winRate['country'].append(i)
    winRate['winrate'].append((len(count[count['home win']] == True) + len(count[count['away win']] == True)) / len(count))
    
winRate = pd.DataFrame(winRate)
#winRate

In [589]:
curRank = rankings[rankings['country_full'].isin(country_list)]
curRank = curRank.loc[curRank['rank_date'] == '2022-10-06'][['country_full', 'rank']]
curRank = curRank.sort_values('country_full')
wc_score['current_rank']= curRank['rank'].values.tolist()
wc_score['avgRank'] = round(avgRank['rank'], 2)
wc_score['winRate'] = round(winRate['winrate'], 2)
wc_score = wc_score[['country_name', 'current_rank', 'avgRank', 'GD', 'winRate']]
wc_score = wc_score.sort_values('current_rank').reset_index().drop(['index'], axis = 1)
wc_score

,country_name,current_rank,avgRank,GD,winRate
0,Brazil,1,5.06,1.63,0.47
1,Belgium,2,4.49,1.40,0.65
2,Argentina,3,4.47,0.97,0.49
3,France,4,9.69,0.86,0.47
4,England,5,9.68,1.28,0.54
5,Spain,7,6.72,1.18,0.55
6,Netherlands,8,14.15,0.98,0.54
7,Portugal,9,6.43,1.05,0.53
8,Denmark,10,23.78,0.75,0.51
9,Germany,11,5.98,1.17,0.56


In [590]:
corr = wc_score.corr()
corr.style.background_gradient()

,current_rank,avgRank,GD,winRate
current_rank,1.000000,0.827994,-0.675269,-0.299605
avgRank,0.827994,1.000000,-0.502896,-0.313125
GD,-0.675269,-0.502896,1.000000,0.383654
winRate,-0.299605,-0.313125,0.383654,1.000000


In [600]:
matches[['home_team', 'away_team', 'home_score', 'away_score']]
# matches[matches['home_team', 'away_team', 'home_score', 'away_score']]
# matches['score_difference'] = matches['home_score'] - matches['away_score']
# matches['is_won'] = matches['score_difference'] > 0

,home_team,away_team,home_score,away_score
3,Qatar,United Arab Emirates,1.0,3.0
7,Saudi Arabia,Iraq,0.0,2.0
8,Tunisia,Ethiopia,1.0,1.0
10,Qatar,Oman,2.0,1.0
12,Zambia,Morocco,0.0,0.0
...,...,...,...,...
8903,Qatar,Chile,2.0,2.0
8904,Saudi Arabia,USA,0.0,0.0
8908,Portugal,Spain,0.0,1.0
8909,Switzerland,Czech Republic,2.0,1.0


## 3. Modeling

In [570]:
# from xgboost import XGBClassifier
# from sklearn import ensemble
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [571]:
X, y = wc_score.loc[:,['current_rank', 'avgRank', 'GD', 'winRate']], matches['is_won']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [32, 3158]

In [ ]:
승리했을때  미치는 상관계수 찾기
어차피 월드컵은 카타르에서 열리므로 모두 어웨이경기로 여김
카타르는 16강 확률이 적으므로 배제